<a href="https://colab.research.google.com/github/babujyan/ACA2021/blob/master/ArtLabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="markdown-google-sans">

## **Getting started**
</div>

The document you are reading is not a static web page, but an interactive environment called a **Colab notebook** that lets you write and execute code.

For example, here is a **code cell** with a short Python script that computes a value, stores it in a variable, and prints the result:

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


To execute the code in the above cell, select it with a click and then either press the play button to the left of the code, or use the keyboard shortcut "Command/Ctrl+Enter". To edit the code, just click the cell and start editing.

Variables that you define in one cell can later be used in other cells:

In [2]:
!unzip  gdrive/MyDrive/archive.zip

Streaming output truncated to the last 5000 lines.
  inflating: Human Action Recognition/train/Image_5499.jpg  
  inflating: Human Action Recognition/train/Image_55.jpg  
  inflating: Human Action Recognition/train/Image_550.jpg  
  inflating: Human Action Recognition/train/Image_5500.jpg  
  inflating: Human Action Recognition/train/Image_5501.jpg  
  inflating: Human Action Recognition/train/Image_5502.jpg  
  inflating: Human Action Recognition/train/Image_5503.jpg  
  inflating: Human Action Recognition/train/Image_5504.jpg  
  inflating: Human Action Recognition/train/Image_5505.jpg  
  inflating: Human Action Recognition/train/Image_5506.jpg  
  inflating: Human Action Recognition/train/Image_5507.jpg  
  inflating: Human Action Recognition/train/Image_5508.jpg  
  inflating: Human Action Recognition/train/Image_5509.jpg  
  inflating: Human Action Recognition/train/Image_551.jpg  
  inflating: Human Action Recognition/train/Image_5510.jpg  
  inflating: Human Action Recognition/

In [21]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 33.0 MB/s 


In [22]:
from torch.utils.data import Dataset
import glob
import torch
import numpy as np
import cv2
import os
from pathlib import Path
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchmetrics.functional import accuracy
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

In [4]:
data_path = "Human Action Recognition"

In [65]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [66]:
cls_id_mapping = {'calling': 0,
 'clapping': 1,
 'cycling': 2,
 'dancing': 3,
 'drinking': 4,
 'eating': 5,
 'fighting': 6,
 'hugging': 7,
 'laughing': 8,
 'listening_to_music': 9,
 'running': 10,
 'sitting': 11,
 'sleeping': 12,
 'texting': 13,
 'using_laptop': 14}


class ImageLoader(Dataset):
    def __init__(self, data_path="Human Action Recognition", mode="train", size=(224,224), transform=None):
        self.data_path = data_path
        self.h, self.w = size
        if mode == "train":
            self.dataset_paths = pd.read_csv(os.path.join(data_path, "train_set.csv"))        
        else:
            self.dataset_paths = pd.read_csv(os.path.join(data_path, "val_set.csv"))        

        self.transform = transform[mode]

    def __len__(self):
        return len(self.dataset_paths)
    
    def __getitem__(self, index):
        image_path = self.dataset_paths.iloc[index].filename
        label = self.dataset_paths.iloc[index].label
        
        img_bgr = cv2.imread(os.path.join(data_path, "train", image_path), 1)
        img_bgr = cv2.resize(img_bgr, (int(self.w), int(self.h)),
                                         interpolation=cv2.INTER_LINEAR)
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

        if self.transform:
            transformed = self.transform(img_rgb)
        
        return transformed, cls_id_mapping[label]
    
    
class DataloaderBuilder:
    def __init__(self, batch_size=32,
                 data_path="Human Action Recognition",
                 mode="train",
                 size=(224,224), 
                 num_workers=2, 
                 transform=None):
        self.batch_size = batch_size
        self.data_path = data_path
        self.mode = mode
        self.size = size
        self.dataset = None
        self.num_workers = num_workers
        self.transform = transform
        
    def build(self):
        self.dataset = ImageLoader(data_path=self.data_path,
                                   mode=self.mode,
                                   size=self.size,
                                   transform=self.transform)
        dataloader = DataLoader(dataset=self.dataset,
                                batch_size=self.batch_size,
                                pin_memory=True, 
                                num_workers=self.num_workers)
        return dataloader

In [70]:
dataset_sizes = {}
dataset_sizes["train"] = 10080
dataset_sizes["val"] = 2520
dataloaders = {x: DataloaderBuilder(batch_size=32, num_workers=2, mode=x, transform=data_transforms).build() for x in ['train', 'val']}

In [71]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs.float())
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds==labels)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [72]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [73]:
model_ft = models.resnet18(pretrained=True)
for i, param in enumerate(model_ft.parameters()):
    if i < 51:
      param.requires_grad = False
  
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, len(cls_id_mapping))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=200)

Epoch 0/199
----------
train Loss: 1.8978 Acc: 0.4054
val Loss: 1.2780 Acc: 0.5984

Epoch 1/199
----------
train Loss: 1.4360 Acc: 0.5464
val Loss: 1.1199 Acc: 0.6317

Epoch 2/199
----------
train Loss: 1.3159 Acc: 0.5819
val Loss: 1.0623 Acc: 0.6556

Epoch 3/199
----------
train Loss: 1.2457 Acc: 0.5999
val Loss: 1.0190 Acc: 0.6659

Epoch 4/199
----------
train Loss: 1.2007 Acc: 0.6107
val Loss: 1.0027 Acc: 0.6694

Epoch 5/199
----------
train Loss: 1.1339 Acc: 0.6380
val Loss: 0.9645 Acc: 0.6857

Epoch 6/199
----------
train Loss: 1.1204 Acc: 0.6448
val Loss: 0.9534 Acc: 0.6921

Epoch 7/199
----------
